# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [124]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re

#import nltk stuff
import nltk

nltk.download(['punkt','stopwords', 'wordnet', 'punkt_tab'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#import sklearn stuff
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [125]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM TAB_DR', engine)
X = df.message
y = df.drop(columns=['id', 'message', 'original', 'genre']) 

#check shapes
print(f"Shape of X: {X.shape} // Shape of Y: {y.shape}")
df.head()

Shape of X: (26028,) // Shape of Y: (26028, 36)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
#check data quality
condition = (y > 1) | (y < 0) | (y == np.nan)
rows = condition.any(axis=1)
rows.sum()

np.int64(0)

In [129]:
index = df[rows].index
index

Index([], dtype='int64')

### 2. Write a tokenization function to process your text data

In [130]:
def tokenize(text):
    #Normalize Text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    #Tokenize Text
    tokens = word_tokenize(text)

    #Remove Stopwords
    words = [t for t in tokens if t not in stopwords.words("english")]

    #Lemmatize
    clean_tokens = []
    for word in words:
        clean_tok = WordNetLemmatizer().lemmatize(word).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

<>:3: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\('
C:\Users\P334257\AppData\Local\Temp\1\ipykernel_23568\1877892729.py:3: SyntaxWarning: invalid escape sequence '\('
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


In [131]:
#Test tokenize function
text = "The first time you see The Second Renaissance it may look boring. Look at it at least twice and definitely watch part 2. It will change your view of the matrix. Are the human people the ones who started the war ? Is AI a bad thing ?"
text = tokenize(text)
print(text)

['the', 'first', 'time', 'see', 'the', 'second', 'renaissance', 'may', 'look', 'boring', '.', 'look', 'least', 'twice', 'definitely', 'watch', 'part', '2', '.', 'it', 'change', 'view', 'matrix', '.', 'are', 'human', 'people', 'one', 'started', 'war', '?', 'is', 'ai', 'bad', 'thing', '?']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [162]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10), n_jobs=10))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [163]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [164]:
#Train pipeline
pipeline.fit(X_train.values, y_train)

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000027258EE4AE0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10),
                                       n_jobs=10))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [165]:
# make the predictions on the test data
y_pred = pipeline.predict(X_test)

In [166]:
# Check for shape consistency
print(f"y_pred Shape: {y_pred.shape}")
print(f"y_test Shape: {y_test.shape}")

y_pred Shape: (6507, 36)
y_test Shape: (6507, 36)


In [167]:
print(classification_report(y_test.values, y_pred, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.87      4938
               request       0.81      0.41      0.54      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.76      0.57      0.65      2714
          medical_help       0.66      0.08      0.14       542
      medical_products       0.77      0.07      0.13       347
     search_and_rescue       0.57      0.04      0.08       192
              security       0.00      0.00      0.00       133
              military       0.68      0.08      0.14       225
           child_alone       0.00      0.00      0.00         0
                 water       0.83      0.22      0.35       430
                  food       0.82      0.50      0.62       731
               shelter       0.83      0.26      0.40       598
              clothing       0.77      0.10      0.18       101
                 money       0.79      

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor 

### 6. Improve your model
Use grid search to find better parameters. 

In [168]:
#get the pipeline params
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000027258EE4AE0>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10),
                         n_jobs=10))],
 'transform_input': None,
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000027258EE4AE0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10),
                       n_jobs=10),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __mai

In [ ]:
cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.